<a href="https://colab.research.google.com/github/rsabilio/ia024-projeto-rag/blob/main/1_extracao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages

In [1]:
!pip install -q llama-index llama-parse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00


# Importing packages

In [45]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
import os
import json
import csv
import re
import pandas as pd
from google.colab import userdata
from llama_parse import LlamaParse

nest_asyncio.apply()

os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get('LLAMA_CLOUD_API_KEY_VAGNER')
GITHUB_TOKEN      = userdata.get('GITHUB_TOKEN')
GITHUB_USER_EMAIL = userdata.get('GITHUB_USER_EMAIL')
GITHUB_USER_NAME  = userdata.get('GITHUB_USER_NAME')

!git config --global user.email '''{GITHUB_USER_EMAIL}'''
!git config --global user.name '''{GITHUB_USER_NAME}'''

root = "/content/ia024-projeto-rag"

# Cloning git repo

In [3]:
# Clonar o repositório GitHub
!git clone https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag.git

Cloning into 'ia024-projeto-rag'...
remote: Enumerating objects: 5786, done.
remote: Counting objects: 100% (791/791), done.
remote: Compressing objects: 100% (284/284), done.
remote: Total 5786 (delta 543), reused 742 (delta 501), pack-reused 4995
Receiving objects: 100% (5786/5786), 1.12 GiB | 26.15 MiB/s, done.
Resolving deltas: 100% (1932/1932), done.
Updating files: 100% (5775/5775), done.


In [4]:
def save_on_git(msg):

    %cd /content/ia024-projeto-rag

    !git remote set-url origin https://{GITHUB_TOKEN}@github.com/rsabilio/ia024-projeto-rag

    !git pull origin main

    !git add .

    !git commit -m '''{msg}'''

    !git push origin main

# Extracting to JSON

In [6]:
def extract_and_save_content(parser, src_dir, dst_dir, log_file):

    # Verifica se o arquivo de log existe
    if not os.path.exists(log_file):
        # Cria o arquivo de log com cabeçalhos se não existir
        with open(log_file, 'w', newline='') as csvfile:
            log_writer = csv.writer(csvfile)
            log_writer.writerow(['file_path', 'num_pages'])  # Cabeçalhos do CSV

    for root, dirs, files in os.walk(src_dir):
        for file in files:
            src_file_path = os.path.join(root, file)
            relative_path = os.path.relpath(root, src_dir)
            dst_file_dir  = os.path.join(dst_dir, relative_path)

            #print(src_file_path)

            # Cria o diretório de destino se não existir
            os.makedirs(dst_file_dir, exist_ok=True)

            # Caminho completo do arquivo de destino
            dst_file_path = os.path.join(dst_file_dir, file)
            dst_file_path = f"{dst_file_path}.json"
            if os.path.exists(dst_file_path):
                print(f"*** {src_file_path}  já processado")
                continue

            # Extrai o conteúdo do arquivo usando LlamaParse
            try:
                print(f"{src_file_path}")
                json_objs = parser.get_json_result(src_file_path)

                with open(dst_file_path, 'w', encoding='utf-8') as f:
                    json.dump(json_objs, f, ensure_ascii=False, indent=4)

                # Conta o número de páginas no conteúdo processado
                num_pages = len(json_objs[0]["pages"])

                # Escreve as informações do log em tempo real
                with open(log_file, 'a', newline='') as csvfile:
                    log_writer = csv.writer(csvfile)
                    log_writer.writerow([src_file_path, num_pages])

            except Exception as e:
                print(f"Erro ao processar o arquivo {src_file_path}: {e}")

# Caminhos de exemplo
src_directory = f"{root}/0-downloaded-files/0-reitoria"
dst_directory = f"{root}/1-files-content/0-reitoria"
log_file_path = f"{root}/log-content-extraction.csv"

parser = LlamaParse(verbose=False)

# TODO: generalizar
if src_directory.endswith('portarias'):
    # Extrai e salva o conteúdo mantendo a estrutura dos diretórios
    for ano in range(2014, 2025):
        print(f"\nProcessing: {ano}")
        src_directory_year = f"{src_directory}/{ano}"
        dst_directory_year = f"{dst_directory}/{ano}"

        print("extracting...")
        extract_and_save_content(parser, src_directory_year, dst_directory_year, log_file_path)

        print("\npushing to git...")
        msg = 'Adicionando JSON das Portarias de Boituva'
        save_on_git(msg)
else:
    print(f"\nProcessing: {src_directory}")

    print("extracting...")
    extract_and_save_content(parser, src_directory, dst_directory, log_file_path)

    print("\npushing to git...")
    msg = 'Adicionando JSON das Organizações didáticas'
    save_on_git(msg)



Processing: /content/ia024-projeto-rag/0-downloaded-files/0-reitoria
extracting...
/content/ia024-projeto-rag/0-downloaded-files/0-reitoria/2018.08.07 - Resolução 62 - Aprova Organização Didática Educação Básica.pdf
/content/ia024-projeto-rag/0-downloaded-files/0-reitoria/ESTATUTO IFSP - VERSÃO VIGENTE - Resolução 08-2014 - - ALERAÇÃO da Resolução 872-2013 .pdf
/content/ia024-projeto-rag/0-downloaded-files/0-reitoria/2022.09.06-Resolução-Normativa-N.10-2022-Altera-Resol.-62-2018-O.D.-Ed.-Básica.pdf
/content/ia024-projeto-rag/0-downloaded-files/0-reitoria/2016.12.06 - Resolução 147 - Aprova Organização Didática Cursos Graduação (Pesquisável).pdf

pushing to git...
/content/ia024-projeto-rag
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
Already up to date.
[main 2e11ae5] Adicionando JSON das Portarias de Boituva
 5 files changed, 12027 insertions(+)
 create mode 100644 "1-files-content/0-reitoria/2016.12.06 - Resolu\303\247\303\243o 147

# Organizing dataset

## Estrutura do texto do Estatuto

* Página 1: Resolução de aprovação de alteração do estatuto

* Página 2 em diante: Texto do Estatuto

* O Estatuto possui Título, Capítulos, Seções, Subseções e Artigos.

* Um Título pode ou não ter capítulos.

* Um Capítulo pode ou não ter seções.

Considerei a estrutura até o nível de Capítulos e coloquei todo o conteúdo do capítulo (seções, subseções e artigos) como um texto só.

### Carregando JSON gerado pelo LlamaParse

In [46]:
json_file = f"{root}/1-files-content/0-reitoria/ESTATUTO IFSP - VERSÃO VIGENTE - Resolução 08-2014 - - ALERAÇÃO da Resolução 872-2013 .pdf.json"
with open(json_file, 'r') as file:
    estatuto = json.load(file)
estatuto

[{'pages': [{'page': 1,
    'text': '      INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNOLOGIA DE SAO PAULOMINISTERIO DA EDUCACAO\n                       RESOLUCAO N."8,DE 4 DE FEVEREIRO DE 2014\n                                                                                   Altera        Estatuto do Instituto\n                                                                                   Federal de Educacao, CienciaTecnologia de Sio Paulo\n                           PRESIDENTE              DO      CONSELHO              SUPERIOR            DO     INSTITUTO\n FEDERAL DE EDUCACAO, CIENCIAE TECNOLOGIA DE SAO PAULO.no USO\n de suasatribuicoes regulamentares e, considerando                             decisao do Conselho Superior na\n reuniao do dia 4 de fevereiro de 2014.\n                      RESOLVE:\n                  Art_        Alterar 0S incisos IVe Vdo Artigo 27 do Estatuto do Instituto\n Federal de Sao Paulo, aprovado pela Resolucao n 51, de 31 de agosto de 2009 e alteradoR

### Gerando texto corrido a partir da página 1 e excluindo o rodapé

In [47]:
pages = estatuto[0]["pages"][1:]
dados = []
pattern = re.compile(r'(\n*\s*(Aprovado pela.*))', re.MULTILINE)

for page in pages:
    text = page['text'].strip()
    text = pattern.split(text)[0]+"\n"
    dados.append(text)

dados

['ESTATUTO\n\n\n                                  TÍTULO I - DA INSTITUIÇÃO\n\n\n                                              CAPÍTULO I\n                            DA NATUREZA E DAS FINALIDADES\n\n\n                  Art. 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E\nTECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo,\ncriado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia\nfederal, vinculada ao Ministério da Educação, detentora de autonomia administrativa,\npatrimonial, financeira, didático-pedagógica e disciplinar.\n§1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625,\nCanindé, São Paulo.\n§2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular\ne multicampi, especializada na oferta de educação profissional e tecnológica nas\ndiferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e\ntecnológicos com a sua prática pedagógica, nos 

In [37]:
with open('/content/ia024-projeto-rag/2-dataset/estatuto.txt', 'w') as file:
    file.writelines(dados)

### Adicionando marcadores para indicar inicio/término de Título/Capítulo

In [48]:
texto_completo = ''.join(dados)
texto_completo

'ESTATUTO\n\n\n                                  TÍTULO I - DA INSTITUIÇÃO\n\n\n                                              CAPÍTULO I\n                            DA NATUREZA E DAS FINALIDADES\n\n\n                  Art. 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E\nTECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo,\ncriado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia\nfederal, vinculada ao Ministério da Educação, detentora de autonomia administrativa,\npatrimonial, financeira, didático-pedagógica e disciplinar.\n§1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625,\nCanindé, São Paulo.\n§2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular\ne multicampi, especializada na oferta de educação profissional e tecnológica nas\ndiferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e\ntecnológicos com a sua prática pedagógica, nos t

In [64]:
import re

# Expressão regular para encontrar linhas que começam com 'Título'
pattern_titulo = r'^\n*\s*(TÍTULO.*)'

# Função para adicionar a marcação antes de linhas que começam com 'Título'
def adicionar_marcacao_titulo(match):
    return '\n>>>>>\n\n' + match.group(1)+". "

texto_marcado = re.sub(pattern_titulo, adicionar_marcacao_titulo, texto_completo, flags=re.MULTILINE)


pattern_capitulo = r'^\n*\s*(CAPÍTULO.*)'

def adicionar_marcacao_capitulo(match):
    return '\n*****\n\n' + match.group(1)

texto_marcado = re.sub(pattern_capitulo, adicionar_marcacao_capitulo, texto_marcado, flags=re.MULTILINE)

print(texto_marcado[:200])

ESTATUTO

>>>>>

TÍTULO I - DA INSTITUIÇÃO. 

*****

CAPÍTULO I
                            DA NATUREZA E DAS FINALIDADES


                  Art. 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E
TECNO


### Colocando o título dos capítulos em uma única linha

In [65]:
# Dividir o texto em linhas
linhas = texto_marcado.split('\n')

# Inicialização de variáveis
agrupando_capitulo = False
capitulo_atual = ""
novo_texto = []

# Regex patterns
capitulo_pattern = re.compile(r'^\s*CAPÍTULO.*')
separador_pattern = re.compile(r'^\s*(SEÇÃO|Art\.)')

for linha in linhas:

    if capitulo_pattern.match(linha):
        # Encontrou uma linha que começa com "CAPÍTULO"
        if agrupando_capitulo:
            novo_texto.append(capitulo_atual.strip())
        capitulo_atual = f"{linha.strip()} -"
        agrupando_capitulo = True

    elif separador_pattern.match(linha):
        # Encontrou uma linha que começa com "SEÇÃO" ou "Art."
        if agrupando_capitulo:
            novo_texto.append(capitulo_atual.strip()+". \n")
            agrupando_capitulo = False
        novo_texto.append(linha.strip())

    else:
        # Parte do conteúdo do capítulo
        if agrupando_capitulo:
            capitulo_atual += ' ' + linha.strip()
        else:
            novo_texto.append(linha.strip())

# Adiciona o último capítulo se ainda estiver agrupando
if agrupando_capitulo:
    novo_texto.append(capitulo_atual.strip()+"\n")

# Juntar as linhas em um novo texto
texto_marcado = '\n'.join(novo_texto)
print(texto_marcado[:200])



ESTATUTO

>>>>>

TÍTULO I - DA INSTITUIÇÃO.

*****

CAPÍTULO I - DA NATUREZA E DAS FINALIDADES. 

Art. 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E
TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro n


### Montando JSON agrupando artigos por capítulo/título



```
{
 titulo: Estatuto
 texto_completo: texto completo extraido do PDF
 partes:[
    {titulo: Título,
     capítulos: [
        {titulo: Título do capítulo,
         artigos: texto com todos os artigos do capítulo
        }
     ]
    },
    {titulo: Título,
     artigos: texto com todos os artigos do título
    }
 ]
}
```




In [66]:
texto_marcado

'ESTATUTO\n\n>>>>>\n\nTÍTULO I - DA INSTITUIÇÃO.\n\n*****\n\nCAPÍTULO I - DA NATUREZA E DAS FINALIDADES. \n\nArt. 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E\nTECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo,\ncriado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia\nfederal, vinculada ao Ministério da Educação, detentora de autonomia administrativa,\npatrimonial, financeira, didático-pedagógica e disciplinar.\n§1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625,\nCanindé, São Paulo.\n§2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular\ne multicampi, especializada na oferta de educação profissional e tecnológica nas\ndiferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e\ntecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os\nfins da legislação educacional:\nI. Reitoria, no endereço citado no §1º

In [73]:
texto_sem_espacos = texto_marcado
texto_sem_espacos = texto_sem_espacos.replace("\n\n>>>>>\n\n", ". ")
texto_sem_espacos = texto_sem_espacos.replace("\n\n*****\n\n", ". ")
texto_sem_espacos = re.sub(r'\n', ' ', texto_sem_espacos)
texto_sem_espacos = re.sub(r'\.\.', '. ', texto_sem_espacos)
texto_sem_espacos = re.sub(r'Art.', 'Art', texto_sem_espacos)
texto_sem_espacos = re.sub(r'\s+', ' ', texto_sem_espacos)
texto_sem_espacos

'ESTATUTO. TÍTULO I - DA INSTITUIÇÃO. CAPÍTULO I - DA NATUREZA E DAS FINALIDADES. Art 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo, criado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia federal, vinculada ao Ministério da Educação, detentora de autonomia administrativa, patrimonial, financeira, didático-pedagógica e disciplinar. §1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625, Canindé, São Paulo. §2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular e multicampi, especializada na oferta de educação profissional e tecnológica nas diferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e tecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os fins da legislação educacional: I. Reitoria, no endereço citado no §1º. II. Campus São Paulo, situado na Rua P

In [75]:
doc_json = {}
doc_json['titulo'] = 'Estatuto'
doc_json['texto_completo'] = texto_completo
doc_json['texto_compacto'] = texto_sem_espacos
doc_json['partes'] = []
titulos = texto_marcado.split('\n\n>>>>>\n\n')
for titulo in titulos:
    titulos_json = {}
    titulo_partes = titulo.split('\n*****\n\n')

    # Se tiver capítulos, colocar os artigos junto de seus capítulos
    if len(titulo_partes) != 1:
        capitulos = titulo_partes[1:]
        titulos_json['titulo'] = titulo_partes[0].replace('\n','')
        titulos_json['capitulos'] = []
        for capitulo in capitulos:
            capitulo_partes = capitulo.split('\n\n')
            artigos = ' '.join(capitulo_partes[1:])
            titulos_json['capitulos'].append({'titulo': capitulo_partes[0].strip(),'artigos': artigos})
    else:
        # Titulos sem capítulos
        titulo_partes = titulo_partes[0].split('\n\n\n')
        titulos_json['titulo']  = titulo_partes[0].replace('\n','')
        artigos                 = ' '.join(titulo_partes[1:])
        titulos_json['artigos'] = artigos


    if len(titulos_json) > 0:
        doc_json['partes'].append(titulos_json)

doc_json['partes']

[{'titulo': 'ESTATUTO', 'artigos': ''},
 {'titulo': 'TÍTULO I - DA INSTITUIÇÃO.',
  'capitulos': [{'titulo': 'CAPÍTULO I - DA NATUREZA E DAS FINALIDADES.',
    'artigos': 'Art. 1º - O INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E\nTECNOLOGIA DE SÃO PAULO – IFSP, com sede e foro na cidade de São Paulo,\ncriado nos termos da Lei nº. 11.892, de 29 de dezembro de 2008, constitui-se em autarquia\nfederal, vinculada ao Ministério da Educação, detentora de autonomia administrativa,\npatrimonial, financeira, didático-pedagógica e disciplinar.\n§1º- O IFSP é domiciliado na sede de sua Reitoria, situada na Rua Pedro Vicente, 625,\nCanindé, São Paulo.\n§2º- O IFSP é uma instituição de educação superior, básica e profissional, pluricurricular\ne multicampi, especializada na oferta de educação profissional e tecnológica nas\ndiferentes modalidades de ensino, com base na conjugação de conhecimentos técnicos e\ntecnológicos com a sua prática pedagógica, nos termos da lei. Tem como sedes, para os\nfins da 

In [76]:
with open('/content/ia024-projeto-rag/2-dataset/estatuto.json', 'w', encoding='utf-8') as f:
    json.dump(doc_json, f, ensure_ascii=False, indent=4)

In [77]:
print("push to git...")
msg = 'Troquei Art. por Art no texto_compactado no JSON do Estatuto processado'
save_on_git(msg)

push to git...
/content/ia024-projeto-rag
From https://github.com/rsabilio/ia024-projeto-rag
 * branch            main       -> FETCH_HEAD
Already up to date.
[main 1666b16] Troquei Art. por Art no texto_compactado no JSON do Estatuto processado
 1 file changed, 1 insertion(+), 1 deletion(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 986 bytes | 986.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/rsabilio/ia024-projeto-rag
   b9712aa..1666b16  main -> main
